In [1]:
from sklearn import datasets, svm, metrics
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
from PIL import Image
import cv2
import os
from IPython.display import clear_output

In [3]:
cwd = os.getcwd()
cwd

'/home/rob/Desktop/lambda/labs/github/Notebooks'

In [7]:
X_ALL = np.load(cwd+'/all_cells_reclassified.npy',allow_pickle=True)
y_ALL = np.load(cwd+'/all_classes.npy',allow_pickle=True)

In [10]:
X_ALL.shape

(67102, 784)

In [9]:
y_ALL = np.where(y_ALL=='ERROR', 0, y_ALL)
y_ALL = y_ALL.astype(int)
X_ALL = X_ALL.reshape(784,X_ALL[0])

/home/rob/anaconda3/envs/omega2020/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


TypeError: only integer scalar arrays can be converted to a scalar index

In [61]:
print(X_ALL.shape)
print(y_ALL.shape)

(5315, 784)
(5315,)


In [62]:
knn = KNeighborsClassifier(n_neighbors=3)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X_ALL, y_ALL, test_size=0.2, random_state=1337)

In [65]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4252, 784)
(1063, 784)
(4252,)
(1063,)


In [66]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [74]:
y_pred = knn.predict(X_test)

(4252, 784)
(1063, 784)
(4252, 1)
(1063, 1)


In [76]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.81      0.84        26
           1       0.98      1.00      0.99       106
           2       1.00      0.99      1.00       111
           3       0.95      1.00      0.98       124
           4       1.00      0.98      0.99       116
           5       0.99      0.99      0.99       110
           6       0.99      1.00      1.00       108
           7       1.00      1.00      1.00       114
           8       0.99      0.95      0.97       111
           9       0.99      0.99      0.99       137

    accuracy                           0.98      1063
   macro avg       0.98      0.97      0.97      1063
weighted avg       0.98      0.98      0.98      1063



In [79]:
pickle.dump(knn, open('KNN_sagemaker_model.sav', 'wb'))

In [49]:
all_cells = np.load('all_cell_arrays_unique.npy')

In [50]:
all_cells.shape

(68766, 784)

In [59]:
all_cells

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [255., 255., 255., ...,   0.,   0.,   0.],
       [255., 255., 255., ...,   0.,   0.,   0.],
       [255., 255., 255., ...,   0.,   0.,   0.]])

In [86]:
all_preds = knn.predict(all_cells)

In [93]:
all_preds[2097]

5

In [98]:
np.save('all_cells_class.npy',all_preds)

In [100]:
all_preds == 2

array([ True,  True,  True, ...,  True,  True,  True])

In [101]:
all_preds

array([0, 0, 0, ..., 0, 0, 0])

In [113]:
def show_images(num):
    mask = (all_preds == num)
    filtered_cells = all_cells[mask]
    np.save(str(num)+"_classified.npy",filtered_cells)
    return filtered_cells

In [ ]:
show_images(1)

In [169]:
def predict_number(num):
    cell_filepath = '/home/rob/Desktop/lambda/labs/github/Notebooks/'+str(num)+'_classified.npy'
    cell_array = np.load(cell_filepath)
    
    list_images = []
    index = []
    for i in range(cell_array.shape[0]):


        pic_array = cell_array[i].reshape(28,28)
        new_im = Image.fromarray(pic_array)

        if new_im.mode != 'RGB':
            new_im = new_im.convert('RGB')
        list_images.append(new_im)
        index.append(i)
    
    return index, list_images
    #for x in list_images:
    #    print(display(x, i))

In [367]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

chunk_size = 10


In [368]:
index, images = predict_number(0)

In [213]:
index = chunks(index,chunk_size)
#images = chunks(images,chunk_size)

In [214]:
for i in index:
    index_list = i
    for k in index_list:
        print(k)
        display(images[k])
    input("Continue?")
    clear_output()

In [180]:
for i in range(2947,2948):
    display(images[i])

In [187]:
display(images[4374])

In [333]:
one_issues = [0,1,2,3,4,5,6,7,8,9,10,11,15,94,101,110,111,113,114,116,119,120,121,122,124,126,127,128,129,134,125,139,144,145,146,147,148,178,271,867,1257,1405,1530,2532,2947, 3562, 3582,2583,3804,3816,3826,3830,3831,3849,4333,4346,4354,4620,7068,7154,7155,7156,3446]
two_issues = [0,1,4541]
three_issues = [0,1,3,4,5,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,30,31,32,33,34,35,37,3840,41,42,43,44,45,47,48,49,50,51,52,54,55,56,57,58,59,60,61,63,64,66,68,70,71,74,76,77,81,82,89,160,161,162,165,166,167,168, 171,173,175,176,177,182,183,186,190,191,192,193,194,197,198,202,211,213,233,238,240,246,247,251,253,260,261,267,269,298,303,307,352,353,354,358,388,389,425,437,440,441,467,468,477,482,483,495,551,641,770,892,926,1048,1050,1617,2218,4517,4518,4519,4523,4527,4530,4534,4535,4537,4538,4539,4540,4542,4545,4548,4552,4572,4617,4739,4753,4761,4769,4770,4772,4838,4848,4849,4867,4872,5166,5185,5367,5368,5621,5829,5887,5936,6937,7025,7299,7302,7301,7318,7319,7320,7321,7324,7327,7331,7332,7336,7442,7343,7344,7345,7346,7352,7356,7358,7361,7363,7367,7369,7372,7376,7406,7407,7413,7423,7467,7541,7656,7657,7672,7673,7677,7678,7682,7685,7687,7689,7694,7699,7709,7711,7713,7715,7716,7719,7721,7724,7736]
four_issues = [1,26,111,112,118,125,127,301,433,677,7405,7407,7409,7410,7411,7416]
five_issues = [683,684,687,687,4241,4293,4297,4317,4549,4666,4667,7301,7302,7303,7304,7306,7307,7309,7299,7319,7320,7350,7373]
six_issues = [118,4075,4374,4604,6367-6379,6518,6519,6877,6888,6889,6883,6890,6891,6942,7157,7497,7498,7500,7501,7513,7519,7520]
seven_issues = [400]
eight_issues = [0,4,163,191,4360-4379,4380,4382,4391,4398,4399,4400,4415,4416,4417,4420,4428,4429,4432,4437,4439,4440,4441,4444,4445,4446,4447,4449,4452,4453,4455,4456,4458,4459,4462,4464,4466,4467,4469,4473,4474,4476,4479,4482,4484,4485,4488,4489,4491,4492,4493,4494,4495,4496,4497,4498,4499,4500,4501,4504,4505,4506,4507,4510,4512,4686,4702,4709,4712]
nine_issues = [124,7530,7531]
zero_issues = [149,152,153,156,213,241,246,242,249,253,257,258,259,262,265,268,273,275,281,282,283,284,285,286,288,290,292,331,354,357,358,359,372,376,377,378,379,381,382,383,385,386,387,389,390,391,392,410,413,417,440,501,532,533,546,576,590,651,650, 677, 710,713,714,745,747,749,752,753,756,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,793,794,795,796,797,798,799,800,801,802,803,805,809,808,810,811,813,814,818,820,826,830,832,837,838,839,841,843,870,885,888,891,896,899,905,911,920,924,925,929,930,937,941,950,957,959,963,969,974,976,977,992,993,996,999,1003,1004,1007,1008,1010,1011,1012,1014,1016,1017,1020,1021,1022,1023,1025,1027,1029,1030,1033,1034,1035,1037,1038,1046,1047,1051,1052,1053,1054,1055,1056,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1087,1088,1089,1090,1091,1093,1095,1096,1097,1098,1099,1100,1102,1103,1004,1005,1006,1007,1108,1109,1110,1111,1112,1113,1114,1115,1117,1118,1122,1130,1131,1143,1146,1149,1150,1151,1152,1155,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227,1228,1229,1230,1231,1232,1233,1234,1235,1236,1237,1238,1239,1240,1241,1242,1243,1244,1245,1246]

In [324]:
array_paths = []
for num in range(0,10):
    apath = '/home/rob/Desktop/lambda/labs/github/Notebooks/'+str(num)+'_classified.npy'
    array_paths.append(apath)

In [325]:
array_paths

['/home/rob/Desktop/lambda/labs/github/Notebooks/0_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/1_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/2_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/3_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/4_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/5_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/6_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/7_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/8_classified.npy',
 '/home/rob/Desktop/lambda/labs/github/Notebooks/9_classified.npy']

In [326]:
error_indexes = [zero_issues,one_issues,two_issues,three_issues,four_issues,five_issues,six_issues,seven_issues,eight_issues,nine_issues]

In [309]:
test_array = np.load(array_paths[9])

In [310]:
testimg = test_array[np.array(error_indexes[9])][0]

In [272]:
pic_array = testimg.reshape(28,28)
new_im = Image.fromarray(pic_array)
if new_im.mode != 'RGB':
    new_im = new_im.convert('RGB')

In [273]:
display(new_im)

In [334]:
a_array.shape

(7512, 784)

In [344]:
for i in range(0,10):
    print("Class: "+str(i))
    e = error_indexes[i]
    a = np.load(array_paths[i])
    m = np.zeros(a.shape[0], dtype=bool)
    m[e] = True
    
    print("Array loaded is size: "+str(a.shape))
    error_array = a[m]
    np.save('resorted_arrays/errors_from_'+str(i)+"_classification.npy", error_array)
    print("Size of the error Array is: "+str(error_array.shape))
    correct_array = a[~m]
    print("Size of the correct Array is: "+str(correct_array.shape))
    np.save("resorted_arrays/correctly_sorted_"+str(i)+".npy",correct_array)

Class: 0
Array loaded is size: (1247, 784)
Size of the error Array is: (339, 784)
Size of the correct Array is: (908, 784)
Class: 1
Array loaded is size: (7157, 784)
Size of the error Array is: (63, 784)
Size of the correct Array is: (7094, 784)
Class: 2
Array loaded is size: (7631, 784)
Size of the error Array is: (3, 784)
Size of the correct Array is: (7628, 784)
Class: 3
Array loaded is size: (7737, 784)
Size of the error Array is: (219, 784)
Size of the correct Array is: (7518, 784)
Class: 4
Array loaded is size: (7419, 784)
Size of the error Array is: (16, 784)
Size of the correct Array is: (7403, 784)
Class: 5
Array loaded is size: (7385, 784)
Size of the error Array is: (22, 784)
Size of the correct Array is: (7363, 784)
Class: 6
Array loaded is size: (7521, 784)
Size of the error Array is: (22, 784)
Size of the correct Array is: (7499, 784)
Class: 7
Array loaded is size: (7625, 784)
Size of the error Array is: (1, 784)
Size of the correct Array is: (7624, 784)
Class: 8
Array lo

In [340]:
import os
os.getcwd()

'/home/rob/Desktop/lambda/labs/github/Notebooks'

In [348]:
load_errors = []
for i in range(0,10):
    path = "/home/rob/Desktop/lambda/labs/github/Notebooks/resorted_arrays/errors_from_"+str(i)+"_classification.npy"
    load_errors.append(path)

In [358]:
all_errors = np.zeros((1,784))
for i in range(0,10):
    new_array = np.load(load_errors[i])
    all_errors = np.concatenate((new_array,all_errors),axis=0)

In [412]:
np.save("All_Errors.npy",all_errors)

In [413]:
def predict_number():
    cell_filepath = '/home/rob/Desktop/lambda/labs/github/Notebooks/All_Errors.npy'
    cell_array = np.load(cell_filepath)
    
    list_images = []
    index = []
    for i in range(cell_array.shape[0]):


        pic_array = cell_array[i].reshape(28,28)
        new_im = Image.fromarray(pic_array)

        if new_im.mode != 'RGB':
            new_im = new_im.convert('RGB')
        list_images.append(new_im)
        index.append(i)
    
    return index, list_images
    #for x in list_images:
    #    print(display(x, i))

In [414]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

chunk_size = 1

In [415]:
index, images = predict_number()
index = chunks(index,chunk_size)

In [417]:
#entries = []
for i in index:
    index_list = i
    for k in index_list:
        #print(k)
        display(images[k])
    value = input("Continue?")
    entries.append(value)
    clear_output()

KeyboardInterrupt: 

In [403]:
final_entries = entries

In [407]:
final_entries = [x if x != 5 else '5' for x in final_entries]

In [408]:
Counter(final_entries)

Counter({'2': 52,
         '5': 149,
         '6': 119,
         '3': 51,
         '8': 149,
         '9': 71,
         '0': 35,
         '1': 46,
         '4': 48,
         '7': 37})

In [419]:
final_entries[0]

'2'

In [423]:
error_class_array = np.array(final_entries).reshape(len(final_entries),)

In [424]:
np.save("All_Error_Classes.npy",error_class_array)

In [439]:
allCells = np.array([])
allCells = allCells.reshape(0,784)
allClass = np.array([])
allClass = allClass.reshape(0,)
for i in range(1,10):
    path = '/home/rob/Desktop/lambda/labs/github/Notebooks/resorted_arrays/correctly_sorted_'+str(i)+".npy"
    array = np.load(path)
    
    number_entries = array.shape[0]
    answer_array = np.repeat(i,number_entries)
    answer_array = answer_array.reshape(array.shape[0],)
    
    allCells = np.concatenate((allCells,array),axis=0)
    allClass = np.concatenate((allClass,answer_array),axis=0)

In [425]:
os.getcwd()

'/home/rob/Desktop/lambda/labs/github/Notebooks'

In [429]:
path = '/home/rob/Desktop/lambda/labs/github/Notebooks/resorted_arrays/correctly_sorted_1.npy'
array = np.load(path)

In [431]:
init_answer = np.zeros(1,)

In [434]:
allArrays = np.array([])

In [441]:
allClass.shape

(67102,)

In [442]:
allCells.shape

(67102, 784)

In [443]:
np.save('all_cells_reclassified.npy',allCells)

In [444]:
np.save('all_classes.npy',allClass)

In [447]:
import random


In [459]:

for x in range(0,10):
    i = random.randint(0,allCells.shape[0])
    test_cell = allCells[i]
    pic_array = test_cell.reshape(28,28)
    new_im = Image.fromarray(pic_array)
    if new_im.mode != 'RGB':
        new_im = new_im.convert('RGB')
    print(allClass[i])
    display(new_im) 
    

9.0


5.0


3.0


4.0


8.0


5.0


5.0


6.0


1.0


8.0


3.0